# Classic Computer Science Problems in Python
## Part 3 Section 1: Graph Frameworks
In this notebook we will explore two different flavors of graphs: weighted and unweighted. Weighted graphs associate a weight with the edges between two vertices. This weight can represent a bunch of different things, but in this particular example, the weights will correspond to the distance (integer value) between two vertices (cities) on a map.


Kicking things off with an unweighted graph, the first step is to define an Edge class that will 'connect' two vertices of generic type V. This means that the vertices could potentially be any type with arbitrary complexity.

In [2]:
from dataclasses import dataclass
from typing import TypeVar, Generic, List, Optional, Callable, Deque, Tuple, Dict

In [3]:
# The dataclass decorator function allows us to skip the __init__ method
# boilerplate and will instantiate instance variables for any variables 
# declared in the type annotations in the class's body
@dataclass
class Edge:
    # u and v are vertices: by convention, 
    # u corresponds to 'from' and v corresponds to the destination
    u: int
    v: int
        
    # returns a new class instance connecting two vertices in reverse
    # to be used with undirected graphs where vertice connections are bi-directional
    # directed graphs can have unidirectional connections between 
    # vertices and this reverse function will not be necessary. 
    def reversed(self) -> 'Edge':
        return Edge(self.v, self.u)
    
    def __str__(self) -> str:
        return f"{self.u} -> {self.v}"
    


In [4]:
V = TypeVar('V')

class Graph(Generic[V]):
    def __init__(self, vertices: List[V] = []) -> None:
        self._vertices: List[V] = vertices
        self._edges: List[List[Edge]] = [[] for _ in vertices]
    
    @property
    def vertex_count(self) -> int:
        return len(self._vertices)
    
    @property
    def edge_count(self) -> int:
        return len(self._edges)
    
    def add_vertex(self, vertex: V) -> int:
        # adds a vertex and returns its index in the vertices array
        self._vertices.append(vertex)
        # adds an empty container to the edges array at the same index
        # as the current vertex so we can have an easy way to update
        # the connections to + from that particular vertex
        self._edges.append([])
        
        # returns the current count - 1 == current vertex index
        return self.vertex_count - 1
    
    def add_edge(self, edge: Edge) -> None:
        # print(self._edges)
        self._edges[edge.u].append(edge)
        # since this graph is undirected, all connections are bi-directional
        self._edges[edge.v].append(edge.reversed())
    
    # convenience method to update the edges by index 
    def add_edge_by_indices(self, u:int, v:int) -> None:
        edge: Edge = Edge(u, v)
        self.add_edge(edge)
        
    # convenience method to add edges by supplying vertices as arguments
    def add_edge_by_vertices(self, first: V, second: V) -> None:
        u: int = self._vertices.index(first)
        v: int = self._vertices.index(second)
        self.add_edge_by_indices(u, v)
        
    # returns the vertex / node at a specific index
    def vertex_at(self, index: int) -> V:
        return self._vertices[index]
    
    # returns the index of a specific vertex using the index method
    def index_of(self, vertex: V) -> int:
        return self._vertices.index(vertex)
    
    # returns a list of all neighbor (connected) nodes / vertices by index
    def neighbors_for_index(self, index: int) -> List[V]:
        # print([e for e in self._edges[index]])
        return list(map(self.vertex_at, [e.v for e in self._edges[index]]))
    
    # returns a list of all neighbor vertices by supplying a vertex argument
    def neighbors_for_vertex(self, vertex: V) -> List[V]:
        return self.neighbors_for_index(self.index_of(vertex))
    
    # returns a list of edges for a specific index
    def edges_for_index(self, index: int) -> List[Edge]:
        return self._edges[index]
    
    # returns a list of edges by getting the index of a specific vertex
    def edges_for_vertex(self, vertex: V) -> List[Edge]:
        return self.edges_for_index(self.index_of(vertex))
    
    # pretty print stuff
    def __str__(self) -> str:
        desc: str = ""
        for i in range(self.vertex_count):
            desc += f"{self.vertex_at(i)} -> {self.neighbors_for_index(i)}\n"
        
        return desc
        
    

In [5]:
locations: List[str] = ["Seattle", "San Francisco", "Los Angeles", 
                         "Riverside", "Phoenix", "Chicago", 
                         "Boston", "New York", "Atlanta", 
                         "Miami", "Dallas", "Houston", 
                         "Detroit", "Philadelphia", "Washington"]
    
city_graph: Graph[str] = Graph(locations)
    
city_graph.add_edge_by_vertices("Seattle", "Chicago")
city_graph.add_edge_by_vertices("Seattle", "San Francisco")  
city_graph.add_edge_by_vertices("San Francisco", "Riverside")  
city_graph.add_edge_by_vertices("San Francisco", "Los Angeles")  
city_graph.add_edge_by_vertices("Los Angeles", "Riverside")  
city_graph.add_edge_by_vertices("Los Angeles", "Phoenix")  
city_graph.add_edge_by_vertices("Riverside", "Phoenix")  
city_graph.add_edge_by_vertices("Riverside", "Chicago")  
city_graph.add_edge_by_vertices("Phoenix", "Dallas")  
city_graph.add_edge_by_vertices("Phoenix", "Houston")
city_graph.add_edge_by_vertices("Dallas", "Chicago")  
city_graph.add_edge_by_vertices("Dallas", "Atlanta")  
city_graph.add_edge_by_vertices("Dallas", "Houston")  
city_graph.add_edge_by_vertices("Houston", "Atlanta")
city_graph.add_edge_by_vertices("Houston", "Miami")  
city_graph.add_edge_by_vertices("Atlanta", "Chicago")  
city_graph.add_edge_by_vertices("Atlanta", "Washington")
city_graph.add_edge_by_vertices("Atlanta", "Miami") 
city_graph.add_edge_by_vertices("Miami", "Washington") 
city_graph.add_edge_by_vertices("Chicago", "Detroit") 
city_graph.add_edge_by_vertices("Detroit", "Boston")
city_graph.add_edge_by_vertices("Detroit", "Washington") 
city_graph.add_edge_by_vertices("Detroit", "New York") 
city_graph.add_edge_by_vertices("Boston", "New York") 
city_graph.add_edge_by_vertices("New York", "Philadelphia") 
city_graph.add_edge_by_vertices("Philadelphia", "Washington")

print(city_graph)


Seattle -> ['Chicago', 'San Francisco']
San Francisco -> ['Seattle', 'Riverside', 'Los Angeles']
Los Angeles -> ['San Francisco', 'Riverside', 'Phoenix']
Riverside -> ['San Francisco', 'Los Angeles', 'Phoenix', 'Chicago']
Phoenix -> ['Los Angeles', 'Riverside', 'Dallas', 'Houston']
Chicago -> ['Seattle', 'Riverside', 'Dallas', 'Atlanta', 'Detroit']
Boston -> ['Detroit', 'New York']
New York -> ['Detroit', 'Boston', 'Philadelphia']
Atlanta -> ['Dallas', 'Houston', 'Chicago', 'Washington', 'Miami']
Miami -> ['Houston', 'Atlanta', 'Washington']
Dallas -> ['Phoenix', 'Chicago', 'Atlanta', 'Houston']
Houston -> ['Phoenix', 'Dallas', 'Atlanta', 'Miami']
Detroit -> ['Chicago', 'Boston', 'Washington', 'New York']
Philadelphia -> ['New York', 'Washington']
Washington -> ['Atlanta', 'Miami', 'Detroit', 'Philadelphia']



### Revisiting BFS
Now that the graph has been built, it should be fairly simple to derive the minimum number of 'hops' it takes to get from one vertex to another. Recalling that the shortest path between to points (eg, start and goal in the maze problems) can be derived using Breadth First Search, some minor modifications to the maze solving BFS algorithm should suffice for this use case. One important thing to note is that this will only work assuming the graph is _unweighted_.

In [7]:
# Needs a data structure to handle graph exploration
class Queue(Generic[V]):
    def __init__(self) -> None:
        self._container: Deque[V] = Deque()
            
    @property
    def empty(self) -> bool:
        return not self._container
    
    def push(self, item: V) -> None:
        self._container.append(item)
    
    def pop(self) -> V:
        return self._container.popleft()
    
    def __repr__(self) -> str:
        return repr(self._container)
    
# Needs a node class to hold state and reference to parent node
class Node(Generic[V]):
    def __init__(self, state: V, parent: Optional['Node']):
        self.state: V = state
        self.parent: Optional['Node'] = parent
            
# Needs a way to trace node path
def node_to_path(node: Node[V]) -> List[V]:
    # initialize the path to final node state (goal vertex)
    path: List[V] = [node.state]
    
    # Until we get back to the start (where parent == None)
    while node.parent is not None:
        # swap pointer for node to its parent
        node = node.parent
        # add the vertex to the list
        path.append(node.state)
    
    # reverse the array since we've added the vertices from end to beginning
    path.reverse()
    return path


# Needs a search algo
def bfs(start: V, goal_check: Callable[[V], bool], successors: Callable[[V], List[V]]) -> Optional[Node[V]]:
    # frontier to explore
    frontier: Queue[Node[V]] = Queue()
    # initialize frontier with starting vertex, no parents on start
    frontier.push(Node(start, None))
    
    # already explored, set of vertices
    explored: Set[V] = {start}
    
    # while there are nodes in the queue
    while not frontier.empty:
        # grab the leftmost node
        current_node: Node[V] = frontier.pop()
        # vertex corresponding to the popped Node
        current_state: V = current_node.state
            
        if goal_check(current_state):
            # return the current node and end the loop
            # grabs the node so we have access to the parent node
            # and will be able to trace back to the start for shortest path
            return current_node
        
        # generates the possible successors from the current vertex
        for child in successors(current_state):
            # skip iteration if the child has been explored
            if child in explored:
                continue
            
            # populates the frontier with child Nodes
            frontier.push(Node(child, current_node))
            
            # tracks which places have been explored
            explored.add(child)
            
    return None

bfs_solution: List[V] = bfs("Detroit", lambda x: x == "San Francisco", city_graph.neighbors_for_vertex)

if bfs_solution is None:
    print('No path found')
else: 
    path: List[V] = node_to_path(bfs_solution)
    print('shortest path between Detroit and San Francisco: ')
    print(path)
    
    
    

shortest path between Detroit and San Francisco: 
['Detroit', 'Chicago', 'Seattle', 'San Francisco']


## Section 2: Weighted Graphs
Given that the unweighted graph does not account for distance between cities (Vertices), the above BFS will only provide the 'shortest' path based off of the number of hops / layovers. To find the 'true' shortest distance between two of these cities (vertices) we will need to incorporate a weighting system to keep track of the their respective distances, (Edge length).


With a weighted graph, we would be able to derive the actual shortest distance between two vertices. In addition, we could ascertain the minimum distance required to connect all 15 cities from the previous unweighted graph implementation. This problem can be solved with Jarnik's algorithm, (otherwise known as Prim's algorithm), and it requires us to subclass Edge and Graph with WeightedEdge and WeightedGraph respectively.

In [9]:
@dataclass
class WeightedEdge(Edge):
    weight: float
        
    def reversed(self) -> 'WeightedEdge':
        return WeightedEdge(self.v, self.u, self.weight)
    
    # Adding a comparison operator since Jarnik's algorithm 
    # is only interested in finding the smallest edge by weight
    def __lt__(self, other: 'WeightedEdge') -> bool:
        return self.weight < other.weight
    
    def __str__(self) -> str:
        return f"{self.u} {self.weight} > {self.v}"
    

In [10]:
V = TypeVar("V")

class WeightedGraph(Generic[V], Graph[V]):
    def __init__(self, vertices: List[V] = []) -> None:
        self._vertices: List[V] = vertices
        self._edges: List[List[WeightedEdge]] = [[] for _ in vertices]
    
    def add_edge_by_indices(self, u: int, v: int, weight: float) -> None:
        edge: WeightedEdge = WeightedEdge(u, v, weight)
        self.add_edge(edge)
        
    def add_edge_by_vertices(self, first: V, second: V, weight: float) -> None:
        u: int = self._vertices.index(first)
        v: int = self._vertices.index(second)
        self.add_edge_by_indices(u, v, weight)
        
    def neighbors_for_index_with_weights(self, index: int) -> List[Tuple[V, float]]:
        distance_tuples: List[Tuple[V, float]] = []
        for edge in self.edges_for_index(index):
            distance_tuples.append((self.vertex_at(edge.v), edge.weight))
        
        return distance_tuples
    
    def __str__(self) -> str:
        desc: str = ""
        for i in range(self.vertex_count):
            desc += f"{self.vertex_at(i)} -> {self.neighbors_for_index_with_weights(i)} \n"
        
        return desc

In [11]:
locations: List[str] = ["Seattle", "San Francisco", "Los Angeles", 
                         "Riverside", "Phoenix", "Chicago", 
                         "Boston", "New York", "Atlanta", 
                         "Miami", "Dallas", "Houston", 
                         "Detroit", "Philadelphia", "Washington"]
    
weighted_city_graph: WeightedGraph[str] = WeightedGraph(locations)
    
weighted_city_graph.add_edge_by_vertices("Seattle", "Chicago", 1737)
weighted_city_graph.add_edge_by_vertices("Seattle", "San Francisco", 678)  
weighted_city_graph.add_edge_by_vertices("San Francisco", "Riverside", 386)  
weighted_city_graph.add_edge_by_vertices("San Francisco", "Los Angeles", 348)  
weighted_city_graph.add_edge_by_vertices("Los Angeles", "Riverside", 50)  
weighted_city_graph.add_edge_by_vertices("Los Angeles", "Phoenix", 357)  
weighted_city_graph.add_edge_by_vertices("Riverside", "Phoenix", 307)  
weighted_city_graph.add_edge_by_vertices("Riverside", "Chicago", 1704)
weighted_city_graph.add_edge_by_vertices("Phoenix", "Dallas", 887)  
weighted_city_graph.add_edge_by_vertices("Phoenix", "Houston", 1015)
weighted_city_graph.add_edge_by_vertices("Dallas", "Chicago", 805)  
weighted_city_graph.add_edge_by_vertices("Dallas", "Atlanta", 721)
weighted_city_graph.add_edge_by_vertices("Dallas", "Houston", 225)
weighted_city_graph.add_edge_by_vertices("Houston", "Atlanta", 702)
weighted_city_graph.add_edge_by_vertices("Houston", "Miami", 968)  
weighted_city_graph.add_edge_by_vertices("Atlanta", "Chicago", 588)  
weighted_city_graph.add_edge_by_vertices("Atlanta", "Washington", 543)
weighted_city_graph.add_edge_by_vertices("Atlanta", "Miami", 604) 
weighted_city_graph.add_edge_by_vertices("Miami", "Washington", 923) 
weighted_city_graph.add_edge_by_vertices("Chicago", "Detroit", 238) 
weighted_city_graph.add_edge_by_vertices("Detroit", "Boston", 613)
weighted_city_graph.add_edge_by_vertices("Detroit", "Washington", 396) 
weighted_city_graph.add_edge_by_vertices("Detroit", "New York", 482) 
weighted_city_graph.add_edge_by_vertices("Boston", "New York", 190) 
weighted_city_graph.add_edge_by_vertices("New York", "Philadelphia", 81) 
weighted_city_graph.add_edge_by_vertices("Philadelphia", "Washington", 123)

print(weighted_city_graph)


Seattle -> [('Chicago', 1737), ('San Francisco', 678)] 
San Francisco -> [('Seattle', 678), ('Riverside', 386), ('Los Angeles', 348)] 
Los Angeles -> [('San Francisco', 348), ('Riverside', 50), ('Phoenix', 357)] 
Riverside -> [('San Francisco', 386), ('Los Angeles', 50), ('Phoenix', 307), ('Chicago', 1704)] 
Phoenix -> [('Los Angeles', 357), ('Riverside', 307), ('Dallas', 887), ('Houston', 1015)] 
Chicago -> [('Seattle', 1737), ('Riverside', 1704), ('Dallas', 805), ('Atlanta', 588), ('Detroit', 238)] 
Boston -> [('Detroit', 613), ('New York', 190)] 
New York -> [('Detroit', 482), ('Boston', 190), ('Philadelphia', 81)] 
Atlanta -> [('Dallas', 721), ('Houston', 702), ('Chicago', 588), ('Washington', 543), ('Miami', 604)] 
Miami -> [('Houston', 968), ('Atlanta', 604), ('Washington', 923)] 
Dallas -> [('Phoenix', 887), ('Chicago', 805), ('Atlanta', 721), ('Houston', 225)] 
Houston -> [('Phoenix', 1015), ('Dallas', 225), ('Atlanta', 702), ('Miami', 968)] 
Detroit -> [('Chicago', 238), ('Bos

### Minimum Spanning Tree
A _minimum spanning tree_ is a tree that connects every vertex in a weighted graph with the minimum total weight. There are many different applications for this: optimizing telecommunications networks, transportation networks, shipping, and utilities. For example, stores and warehouses can be considered vertices with edges representing the distances between them: to derive the most efficient way of connecting these vertices will have tremendous upside for cost savings, (eg. gas for trucks to ferry merchandise between locations).


Using Jarnik's algorithm, (otherwise known as Prim's algorithm), works by dividing a graph into two parts: the vertices in the currently-being-assembled minimum spanning tree and the vertices not yet contained in the minimum spanning tree. Here's a brief outline of its process:
1. Choose an vertex to include in the MST
2. Find the lowest-weighted edge connecting the in-progress MST to vertices not yet in the MST
3. Add the vertex at the end of that minimum edge to the MST
4. Repeat 2 and 3 until every vertex in the graph is part of the MST

The data structure used with Jarnik's algorithm is, similar to the A* pathfinding algorithm, a priority queue. With a priority queue we can ensure that the minimum weighted edge will be popped. One important note, however, is that Jarnik's algorithm will not necessarily work in a graph with directed edges, and will not work in a graph that is not connected.

#### Python Heap Libraries
The following implementation of Priority Queue uses heap push and heap pop to maintain order. From the [documentation](https://docs.python.org/2/library/heapq.html), Heaps are binary trees for which every parent node has a value less than or equal to any of its children. This implementation uses arrays for which heap\[k\] <= heap\[2*k+1\] and heap\[k\] <= heap\[2*k+2\] for all k, counting elements from zero. For the sake of comparison, non-existing elements are considered to be infinite. The interesting property of a heap is that its smallest element is always the root, heap\[0\].

In [12]:
from heapq import heappush, heappop, heapify

# using T because we're storing edges rather than vertices 
T = TypeVar("T")

class PriorityQueue(Generic[T]):
    def __init__(self) -> None:
        self._container: List[T] = []
    
    @property
    def empty(self) -> bool:
        return not self._container
    
    def push(self, item: T) -> None:
        heappush(self._container, item)
    
    def pop(self) -> T:
        return heappop(self._container)
    
    def __repr__(self) -> str:
        return repr(self._container)
    

WeightedPath = List[WeightedEdge] # Type alias for path

# helper function to sum all weights in the path
def total_weight(wp: WeightedPath) -> float:
    return sum([e.weight for e in wp])

def mst(wg: WeightedGraph[V], start: int = 0) -> Optional[WeightedPath]:
    # Breaks if the starting point is invalid
    # more concretely: if the starting point is outside the scope of
    # the list of vertices, the starting point is invalid and we can return None
    if start > (wg.vertex_count - 1) or start < 0:
        return None
    
    # Container to store the edges
    result: WeightedPath = []
    pq: PriorityQueue[WeightedEdge] = PriorityQueue()
    
    # Initialize a list of booleans with length == number of vertices
    visited: List[bool] = [False] * wg.vertex_count
    
    # Convenience function that updates the visited array and populates
    # the priority queue with all edges for that particular index
    # after pruning the edges that have already been explored.
    # Serves the same purpose as the 'successors' method in BFS
    def visit(index: int):
        visited[index] = True
        for edge in wg.edges_for_index(index):
            if not visited[edge.v]:
                pq.push(edge)
    
    # Initializes the priority queue 
    visit(start)
    
    # While there are weighted edges in the priority queue, crawl the graph
    while not pq.empty:
        edge = pq.pop()
        # if the index of the 'to' vertex has already been visited, skip iteration
        if visited[edge.v]:
            continue
        
        result.append(edge)
        visit(edge.v)
    
    return result


def print_weighted_path(wg: WeightedGraph, wp: WeightedPath) -> None:
    for edge in wp:
        print(f"{wg.vertex_at(edge.u)} {edge.weight} > {wg.vertex_at(edge.v)}")
    
    print(f"Total Weight: {total_weight(wp)}")
        
jarnik = mst(weighted_city_graph, start=0)

if jarnik is None:
    print('no solution found')
else:
    print_weighted_path(weighted_city_graph, jarnik)

Seattle 678 > San Francisco
San Francisco 348 > Los Angeles
Los Angeles 50 > Riverside
Riverside 307 > Phoenix
Phoenix 887 > Dallas
Dallas 225 > Houston
Houston 702 > Atlanta
Atlanta 543 > Washington
Washington 123 > Philadelphia
Philadelphia 81 > New York
New York 190 > Boston
Washington 396 > Detroit
Detroit 238 > Chicago
Atlanta 604 > Miami
Total Weight: 5372


### Dijkstra's Algorithm
This algorithm solves for the single-source shortest path problem. From a starting vertex, it returns the lowest weight path to any other vertex on a weighted graph, (BFS for weighted graphs), and also returns the minimum total weight to every other vertex from the starting point.


#### Algorithm Breakdown:
1. Add the starting vertex to a priority queue
2. Pop the closest vertex from the priority queue
3. look at all the neighbors connected to that vertex: if they have not been recorded, or if the edge offers a shorter path than had previously been recorded, record the edge and add the new vertex to the priority queue
4. Repeat 2 and 3 until priority queue is empty
5. Return shortest distance to every vertex from the starting vertex and the path to get to each


In order to implement this algorithm, we will need to implement a new Node class (DijkstraNode) that allows us to keep track of the costs associated with each explored vertex for the purposes of comparison.

In [17]:
@dataclass
class DijkstraNode:
    vertex: int
    distance: float
    
    def __lt__(self, other: 'DijkstraNode') -> bool:
        return self.distance < other.distance
    
    def __eq__(self, other: 'DijkstraNode') -> bool:
        return self.distance == other.distance
    
def dijkstra(wg: WeightedGraph[V], root: V) -> Tuple[List[Optional[float]], Dict[int, WeightedEdge]]:
    # Starting point is the int index of the root vertex
    first: int = wg.index_of(root)
    
    # Initialize an array of length == number of vertices with values == None
    distances: List[Optional[float]] = [None] * wg.vertex_count
        
    # Set the distance of the root node index to 0 since there is 
    # no distance between the starting point and itself
    distances[first] = 0
    
    # Initialize the path dictionary and priority queue
    path_dict: Dict[int, WeightedEdge] = {}
    pq: PriorityQueue[DijkstraNode] = PriorityQueue()
    
    # Push the root node into the priority queue
    pq.push(DijkstraNode(first, 0))
    
    # While there are nodes in the queue
    while not pq.empty:
        # get the 'from' vertex index by popping the DijkstraNode with the 
        # smallest distance from the priority queue
        u: int = pq.pop().vertex
        
        # print("distances array: {} target index: {}".format(distances, u))
        # starting distance (from root) updated on each iteration of popping from PQ 
        dist_u: float = distances[u]
        
        # for each edge at index u, which is taken from 
        # the vertex attribute of the popped DijkstraNode
        for we in wg.edges_for_index(u):
            # points dist_v to the distances array at the 'to' index of the weighted edge
            dist_v: float = distances[we.v]
            
            # Checks if the distance has been measured yet
            # checks if the distance is greater than the sum of 
            # the current WeightedEdge's weight (length) and the distance
            # from the root at the current DijkstraNode's vertex index
            if dist_v is None or dist_v > we.weight + dist_u:
                # if the sum of the WeightedEdge's weight and distance from root at
                # the current vertex is less then the distance previously stored in the array
                # update the distances array at the 'to' index to be the the new sum
                distances[we.v] = we.weight + dist_u
                
                # Update the path dictionary at the 'to' index to be the current WeightedEdge
                path_dict[we.v] = we
                
                # Add a new DijkstraNode to the PQ with the 'to' index as the new root
                # and the distance value equal to the sum of the length of the 
                # edge to get there and the distance from the original root
                pq.push(DijkstraNode(we.v, we.weight + dist_u))
    
    return distances, path_dict

def distance_array_to_vertex_dict(wg: WeightedGraph[V], 
                                  distances: List[Optional[float]]) -> Dict[V, Optional[float]]:
    distance_dict: Dict[V, Optional[float]] = {}
    for i in range(len(distances)):
        # Sets key of dict to be the vertex at index i
        # Sets value of dict to be the calculated distance at index i
        distance_dict[wg.vertex_at(i)] = distances[i]
    
    return distance_dict

def path_dict_to_path(start: int, end: int, path_dict: Dict[int, WeightedEdge]) -> WeightedPath:
    if len(path_dict) == 0:
        return []
    edge_path: WeightedPath = []
    
    # set e to the end (target) of the path dictionary and add to edge path array
    e: WeightedEdge = path_dict[end]
    edge_path.append(e)
    # as long as the 'from' index is not == the start index
    while e.u != start:
        # set e to be the path dict value of the edge's 'from' index and add to edge path array
        e = path_dict[e.u]
        edge_path.append(e)
    
    return list(reversed(edge_path))
                
    
distances, path_dict = dijkstra(weighted_city_graph, "Detroit")
name_distance: Dict[str, Optional[int]] = distance_array_to_vertex_dict(weighted_city_graph, distances)

print("Distances from Detroit:")
for key, value in name_distance.items():
    print(f"{key} : {value}")
print(" ")

print("Shortest path from Detroit to San Francisco: ")
path: WeightedPath = path_dict_to_path(weighted_city_graph.index_of("Detroit"), 
                                       weighted_city_graph.index_of("San Francisco"),
                                       path_dict)
    
print_weighted_path(weighted_city_graph, path)

Distances from Detroit:
Seattle : 1975
San Francisco : 2328
Los Angeles : 1992
Riverside : 1942
Phoenix : 1930
Chicago : 238
Boston : 613
New York : 482
Atlanta : 826
Miami : 1319
Dallas : 1043
Houston : 1268
Detroit : 0
Philadelphia : 519
Washington : 396
 
Shortest path from Detroit to San Francisco: 
Detroit 238 > Chicago
Chicago 1704 > Riverside
Riverside 386 > San Francisco
Total Weight: 2328
